In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import uuid

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')

LUSID Environment Initialised


![Initailise LUSID](img/multiplecurrencies-initialise.gif)

## 1) Set up our Scope

In [2]:
scope_id = str(uuid.uuid4())[:4]
scope_name = 'UK_High_Growth_Equities_Fund'
scope = '{}_{}'.format(scope_name, scope_id)
print (scope)

UK_High_Growth_Equities_Fund_80a2


![Scope](img/multiplecurrencies-scope.gif)

## 2) Load Instruments

In [3]:
instrument_universe = pd.read_csv('data/multiplecurrencies-instruments.csv')
instrument_universe.head(n=10)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,USTreasury_2.00_2021,imd_34535347,USD,US912828U816,BBG00FN3B5K8,BERLIN,united_states_america,T 2 12/31/21,govt,us_government,2.000
3,USTreasury_6.875_2025,imd_34534539,USD,US912810EV62,BBG000DQQNJ8,NEW YORK,united_states_america,T 6.875 08/15/25,govt,us_government,6.875


In [4]:
# Initialise our dictionary to hold our instrument definitions
instrument_definitions = {}

# Iterate over our instrument universe
for index, instrument in instrument_universe.iterrows():

    # Create our ISIN property using the default ISIN property definition
    isin_property = models.InstrumentProperty(
        key='Instrument/default/Isin',
        value=models.PropertyValue(
            label_value=instrument['isin']))
    # Create our ticker property using the default ticker property definition
    ticker_property = models.InstrumentProperty(
        key='Instrument/default/Ticker',
        value=models.PropertyValue(
            label_value=instrument['ticker']))

    # Create the definition for our instrument
    instrument_definitions[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers={'Figi': instrument['figi']},
        properties=[
            isin_property,
            ticker_property
        ]
    )

# Call LUSID to upsert our instrument defintions
response = client.upsert_instruments(requests=instrument_definitions)
# Pretty print the response from LUSID
prettyprint.instrument_response(response, identifier='Figi')

Instrument Successfully Upserted: USTreasury_2.00_2021
Figi ID: BBG00FN3B5K8
LUSID Instrument ID: LUID_0BU9BXC2


Instrument Successfully Upserted: USTreasury_6.875_2025
Figi ID: BBG000DQQNJ8
LUSID Instrument ID: LUID_S5F04ES0


Instrument Successfully Upserted: Amazon_Nasdaq_AMZN
Figi ID: BBG000BVPXP1
LUSID Instrument ID: LUID_I6M8YNQN


Instrument Successfully Upserted: Apple_Nasdaq_AAPL
Figi ID: BBG000B9XVV8
LUSID Instrument ID: LUID_K8YYP5FU


4  instruments upserted successfully
0  instrument upsert failures


![Instruments](img/multiplecurrencies-instruments.gif)

## 3) Create our Base Fund Portfolio

In [5]:
base_portfolio_name = '{}_base_fund'.format(scope_name)
print (base_portfolio_name)

UK_High_Growth_Equities_Fund_base_fund


In [6]:
# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=365))

# Create the request to add our portfolio
transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=base_portfolio_name,
    code=base_portfolio_name,
    base_currency='GBP',
    description='The portfolio to hold our base fund',
    created=portfolio_creation_date)

# Call LUSID to create our portfolio
portfolio_response = client.create_portfolio(
    scope=scope,
    create_request=transaction_portfolio_request)

# Pretty print the response from LUSID
prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_80a2
Code: UK_High_Growth_Equities_Fund_base_fund
Portfolio Effective From: 2018-02-19 11:48:04.711367+00:00
Portfolio Created On: 2019-02-19 11:48:04.896497+00:00



![Base-Fund](img/multiplecurrencies-basefundportfolio.gif)

## 4) Set our Holdings

In [7]:
#Import our holdings
holdings = pd.read_csv('data/multiplecurrencies-holdings.csv')
holdings.head()

,instrument_name,client_internal,isin,figi,quantity,unit_cost,total_cost,currency
0,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,5000,1550.00,7750000.00,USD
1,Apple_Nasdaq_AAPL,imd_35345345,US0378331005,BBG000B9XVV8,49567,190.00,9417730.00,USD
2,USTreasury_2.00_2021,imd_34535347,US912828U816,BBG00FN3B5K8,121543,99.25,12063142.75,USD
3,USTreasury_6.875_2025,imd_34534539,US912810EV62,BBG000DQQNJ8,98444,140.98,13878635.12,USD


In [8]:
holding_adjustments = []

holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=3)

for row in holdings.iterrows():
    
    holding = row[1]
    
    holding_adjustments.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                    'Instrument/default/Figi': holding['figi']},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['total_cost'],
                            currency=holding['currency']),
                        portfolio_cost=holding['total_cost'],
                        price=holding['unit_cost'])
                ]
        )
    )
    
# Call LUSID to set our holdings
response = client.set_holdings(
    scope=scope,
    code=base_portfolio_name,
    effective_at=holdings_effective_date,
    holding_adjustments=holding_adjustments)

prettyprint.set_holdings_response(response, scope, base_portfolio_name)

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_80a2
Code: UK_High_Growth_Equities_Fund_base_fund
Holdings Effective From: 2019-02-16 11:48:05.088994+00:00
Holdings Created On: 2019-02-19 11:48:05.800086+00:00



![Base-Fund-Hodlings](img/multiplecurrencies-basefundportfolioholdings.gif)

## 5) Add Corporate Actions

In [9]:
corporate_action_source_id = '{}_corporate_action_source'.format(base_portfolio_name)

corporate_action_request = models.CreateCorporateActionSourceRequest(
    scope=scope,
    code=corporate_action_source_id)

response = client.create_corporate_action_source(
    request=corporate_action_request)
    
print (response)

{'additional_properties': {}, 'href': None, 'id': <lusid.models.resource_id.ResourceId object at 0x7f593b9d42e8>, 'version': None, 'links': [<lusid.models.link.Link object at 0x7f5964070c88>]}


![Corporate-Action-Source](img/multiplecurrencies-corporateactionsource.gif)

In [10]:
corporate_actions = pd.read_csv('data/multiplecurrencies-corporateactions.csv')
corporate_actions.head()

,code,announcement_date,execution_date,record_date,payment_date,input_instrument_figi,input_instrument_internal,input_units_factor,input_cost_factor,output_instrument_figi,output_instrument_internal,output_units_factor,output_cost_factor,dividend_yield,action_description
0,5943592342,2019-01-29 12:04:20.970603+00:00,2019-02-08 12:04:20.970603+00:00,2019-02-11 12:04:20.970603+00:00,2019-02-14 12:04:20.970603+00:00,BBG000B9XVV8,imd_34535347,1,1,NaN,CCY_GBP,2,2,0.0017,dividend


In [11]:
actions = []

for row in corporate_actions.iterrows():
    
    action = row[1]
    
    luid = client.get_instrument(
        identifier_type='Figi',
        identifier=action['input_instrument_figi']).lusid_instrument_id
    
    transition = models.CorporateActionTransition(
        input_transition=models.CorporateActionTransitionComponent(
            instrument_uid=luid,
            units_factor=1,
            cost_factor=0),
        output_transitions=[
            models.CorporateActionTransitionComponent(
            instrument_uid=action['output_instrument_internal'],
            units_factor=action['dividend_yield'],
            cost_factor=0),
        ])
    
    actions.append(
        models.CreateCorporateAction(
            corporate_action_code=action['code'],
            announcement_date=datetime.now(pytz.UTC)+timedelta(days=1),
            ex_date=datetime.now(pytz.UTC)+timedelta(days=1),
            record_date=datetime.now(pytz.UTC)+timedelta(days=1),
            payment_date=datetime.now(pytz.UTC)+timedelta(days=1),
            transitions=[transition])
    )

response = client.batch_upsert_corporate_actions(
    scope=scope,
    code=corporate_action_source_id,
    actions=actions)

print (response)

{'additional_properties': {}, 'href': None, 'values': {'5943592342': <lusid.models.corporate_action.CorporateAction object at 0x7f593b97be80>}, 'failed': {}, 'links': [<lusid.models.link.Link object at 0x7f593b9703c8>]}


In [12]:
response = client.upsert_portfolio_details(
    scope=scope,
    code=base_portfolio_name,
    details=models.CreatePortfolioDetails(
        corporate_action_source_id=models.ResourceId(
            scope=scope,
            code=corporate_action_source_id),
        base_currency='GBP'))

print (response)

{'additional_properties': {}, 'href': 'https://api.lusid.com/api/transactionportfolios/UK_High_Growth_Equities_Fund_80a2/UK_High_Growth_Equities_Fund_base_fund/details?effectiveAt=2019-02-19T11%3A48%3A06.8906540%2B00%3A00&asAt=2019-02-19T11%3A48%3A07.4673550%2B00%3A00', 'origin_portfolio_id': <lusid.models.resource_id.ResourceId object at 0x7f593b96ae10>, 'version': <lusid.models.version.Version object at 0x7f593b9da748>, 'base_currency': 'GBP', 'links': [<lusid.models.link.Link object at 0x7f593b9d4748>]}


![Corporate-Actions](img/multiplecurrencies-corporateactions.gif)

## 6) Value our Base Fund Portfolio

### a) Analytic store

In [13]:
prices = pd.read_csv('data/multiplecurrencies-prices.csv')
prices.head()

,time,price,type,currency,instrument_name,instrument_figi,instrument_internal
0,18/02/2019,1622.65,close,USD,Amazon_Nasdaq_AMZN,BBG000BVPXP1,imd_34634534
1,18/02/2019,170.80,close,USD,Apple_Nasdaq_AAPL,BBG000B9XVV8,imd_34535347
2,18/02/2019,98.00,close,USD,USTreasury_2.00_2021,BBG00FN3B5K8,imd_34535347
3,18/02/2019,142.36,close,USD,USTreasury_6.875_2025,BBG000DQQNJ8,imd_34534539
4,17/02/2019,1619.23,close,USD,Amazon_Nasdaq_AMZN,BBG000BVPXP1,imd_34634534


In [36]:
times = prices['time'].unique()

for time in times:
    quote_date = datetime.strptime(time, '%d/%m/%Y')

    # Create analytics store request
    analytics_store_request = models.CreateAnalyticStoreRequest(
        scope=scope,
        date_property=quote_date)

    # Call LUSID to create our analytics store
    #client.create_analytic_store(request=analytics_store_request)
    
    instrument_analytics = []
    
    i=0
    
    for row, quote in prices.loc[prices['time'] == time].iterrows():
        i+=1
        if i <3:
            continue 
        
        luid = client.get_instrument(
            identifier_type='Figi',
            identifier=quote['instrument_figi']).lusid_instrument_id
        print (luid)
        instrument_analytics.append(
            models.InstrumentAnalytic(
                instrument_uid=luid,
                value=quote['price']))
        
    response = client.set_analytics(
        scope=scope,
        year=quote_date.year,
        month=quote_date.month,
        day=quote_date.day,
        data=instrument_analytics)
    
    print (response)
    

LUID_0BU9BXC2
LUID_S5F04ES0
{'additional_properties': {}, 'key': <lusid.models.analytic_store_key.AnalyticStoreKey object at 0x7f593b6caa20>, 'href': 'https://api.lusid.com/api/analytics/UK_High_Growth_Equities_Fund_80a2/2019/2/18', 'links': None}
LUID_0BU9BXC2
LUID_S5F04ES0
{'additional_properties': {}, 'key': <lusid.models.analytic_store_key.AnalyticStoreKey object at 0x7f593b6db898>, 'href': 'https://api.lusid.com/api/analytics/UK_High_Growth_Equities_Fund_80a2/2019/2/17', 'links': None}


![Analytic-Stores](img/multiplecurrencies-analyticstores.gif)

### b) Aggregation

In [37]:
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=scope,
        code='default'),
        effective_at=datetime.now(pytz.UTC),
        metrics=[
            models.AggregateSpec(key='Holding/default/SubHoldingKey',
            op='Value'),
            models.AggregateSpec(key='Instrument/default/Name',
            op='Value'),
            models.AggregateSpec(key='Holding/default/Units',
            op='sum'),
            models.AggregateSpec(key='Holding/default/Cost',
            op='sum'),
            models.AggregateSpec(key='Holding/default/PV',
            op='sum'),
        ],
        group_by=[
            'Holding/default/SubHoldingKey'
        ])

response = client.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_name,
    request=aggregation_request)

print(response.data)

ErrorResponseException: A price was not found for instrument

## 7) Securitise our Based Fund Portfolio

a) Securitiese with x units

In [17]:
# Create our request to define a new property
property_request = models.CreatePropertyDefinitionRequest(
    domain='Instrument',
    scope=scope,
    code='total_circulation',
    value_required=False,
    display_name='total_circulation',
    data_type_id=models.ResourceId(scope='default', code='number'))

# Call LUSID to create our new property
response = client.create_property_definition(definition=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
circulation_property_key = response.key

# Pretty print our key
prettyprint.heading('Circulation Property Key', circulation_property_key)

Circulation Property Key: Instrument/UK_High_Growth_Equities_Fund_80a2/total_circulation


![Analytic-Stores](img/multiplecurrencies-circulationproperty.gif)

In [18]:
instrument_definition = models.InstrumentDefinition(
    name=base_portfolio_name,
    identifiers={'ClientInternal': base_portfolio_name},
    properties=[models.InstrumentProperty(
        key=circulation_property_key,
        value=models.PropertyValue(
            metric_value=models.MetricValue(
                value=400.000
                )))],
    look_through_portfolio_id=models.ResourceId(
        scope=scope,
        code=base_portfolio_name))

response = client.upsert_instruments(requests={'look-through':instrument_definition}
    
    )

In [19]:
print (response.values)

{'look-through': <lusid.models.instrument.Instrument object at 0x7f593b775a90>}


![Analytic-Stores](img/multiplecurrencies-securitisebase.gif)

b) Calculalate price from aggregation and store in quotes store

In [28]:
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=scope,
        code='default'),
        effective_at=datetime.now(pytz.UTC),
        metrics=[
            models.AggregateSpec(key='Holding/default/Cost',
            op='sum'),
            models.AggregateSpec(key='Holding/default/PV',
            op='sum'),
        ],
        group_by=[
            'Portfolio/default/Name'
        ])

response = client.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_name,
    request=aggregation_request)

total_value = response.data[0]['Sum(Holding/default/PV)']
print (response.data)

[{'Sum(Holding/default/Cost)': 43109507.87, 'Sum(Holding/default/PV)': 42789590.4}]


In [30]:
instrument_analytics = []

quote_date = datetime.now(pytz.UTC) - timedelta(days=1)

inst = client.get_instrument(
            identifier_type='ClientInternal',
            identifier=base_portfolio_name,
            instrument_property_keys=[circulation_property_key])

luid = inst.lusid_instrument_id
total_circulation = inst.properties[0].value
price = total_value / total_circulation

instrument_analytics.append(
    models.InstrumentAnalytic(
        instrument_uid=luid,
        value=price))

response = client.set_analytics(
        scope=scope,
        year=quote_date.year,
        month=quote_date.month,
        day=quote_date.day,
        data=instrument_analytics)

{'additional_properties': {}, 'key': <lusid.models.analytic_store_key.AnalyticStoreKey object at 0x7f593b78aa90>, 'href': 'https://api.lusid.com/api/analytics/UK_High_Growth_Equities_Fund_80a2/2019/2/18', 'links': None}


## 8) Create Foreign Currency Portfolios

## 9) Add Base Fund Units

## 10) Add Hedging Transactions

## 11) Value Currency Portfolios

## 12) Securitise Currency Portfolios

## 13) Create Share Class Portfolios

## 15) Add Currency Fund Holdings

## 16) Value Each Share Class

## 17) Add transactions to end portfolio

## 18) Value each portfolio (bubble up)